In [2]:

import pandas as pd
root = "/Users/k23030440/github_projects/S.-Pombe-MLPs/"


In [3]:
#read in data

#go terms for orthology analysis from: Supplementary material/annotation_orthogroup_genes.xlsx

go_terms = pd.read_csv(root + "internal data/go_terms_formatted.csv",header=1)

#omics data sources
omics_sources = pd.read_excel(root + "internal data/external data sources.xlsx")

#floc_cor_genes /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Analysis of Clement-Ziza RNA-seq/filter_corr.csv
floc_cor_genes = pd.read_csv(root + "Bence folder/Analysis of Clement-Ziza RNA-seq/filter_corr.csv", index_col=0)

#structure_sequence_homology /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Structure_and_Sequence_comparisons/final_analysis.csv
structure_sequence_homology = pd.read_csv(root + "Bence folder/Structure_and_Sequence_comparisons/final_analysis.csv")

#srb11 DE genes /Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Analysis of Clement-Ziza RNA-seq/DESeq2/DE_results.csv
srb11_DE_genes = pd.read_csv(root + "Bence folder/Analysis of Clement-Ziza RNA-seq/DESeq2/DE_results.csv")
#rename first col to Systematic name
srb11_DE_genes.rename(columns={srb11_DE_genes.columns[0]:"Systematic name"}, inplace=True)


#segregant genotypes/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/internal data/updated_genotype_matrix_final_pos.tsv
segregant_genotypes = pd.read_csv(root + "internal data/updated_genotype_matrix_final_pos.tsv", sep="\t")


wi_adh_across_media = pd.read_excel(root + "Bence folder/Image processing - Adhesion assay/condition_plot_df.xlsx",index_col=0)


#plasmids '/Users/k23030440/github_projects/S.-Pombe-MLPs/Supplementary material/plasmid_table.xlsx'
plasmids=pd.read_excel(root + "Supplementary material/plasmid_table.xlsx")

#screen_raw from root + "internal data/Deletion screen raw results.csv
screen_raw = pd.read_csv(root + "internal data/Deletion screen raw results.csv")

#screen enrichment from root + Bence folder/Angeli/angeli/bioneer_angeli_results_recent.txt
screen_enrichment = pd.read_csv(root + "Bence folder/Angeli/angeli/bioneer_angeli_results_recent.txt", sep="\t")

#screen_final root + "Bence folder/Bioneer screen/final_results_screen.csv"
screen_final = pd.read_csv(root + "Bence folder/Bioneer screen/final_results_screen.csv")


#orthology, first col is index
orthology = pd.read_csv(root + "Bence folder/Orthologs/final_orthology_database_new.csv", index_col=0)

#mediator /Users/k23030440/S.-Pombe-MLPs - most recent/Bence folder/Bioneer screen/final_results_mediator.csv
mediator = pd.read_csv(root + "Bence folder/Bioneer screen/final_results_mediator.csv")

#variants S.-Pombe-MLPs/Bence folder/Looking for variants in wild isolates/snp_hunting.csv
variants = pd.read_csv(root + "Bence folder/Looking for variants in wild isolates/snp_hunting.csv")


In [4]:
# Import the necessary libraries
import pandas as pd

# Define the file path for the output xlsx file
output_file = root + "Supplementary material/Supplementary table.xlsx"

# Create a Pandas Excel writer using the output file path
writer = pd.ExcelWriter(output_file, engine='xlsxwriter')

# Define a dictionary containing the dataframes and their corresponding sheet names
dataframes = {
    '1_go_fypo_terms': go_terms,
    '2_structure_sequence_homology': structure_sequence_homology,
    '3_omics_sources': omics_sources,
    '4_segregant_genotypes': segregant_genotypes,
    '5_orthology_relationships': orthology,
    '6_nat_iso_adhesion_across_media' : wi_adh_across_media,
    '7_floc_cor_genes': floc_cor_genes,
    '8_mediator_results': mediator,
    '9_srb11_DE_genes': srb11_DE_genes,
    '10_Screen_raw_results': screen_raw,
    '11_Screen_enrichment': screen_enrichment,
    '12_Screen_final_results': screen_final,
    '13_Variants_in_candidate_genes': variants,
    '14_Plasmids': plasmids
}

descriptions = [
    "#Summary of the GO terms, FYPO terms, and phenotype keywords used for the orthology analysis.",
    "#Summary of results from the structure and sequence homology analysis in a tidy format, ready for plotting.\n",
    "#Summary of omics data sources used in the study.",
    "#Table of segregant genotypes used in the QTL analysis. Y0036 is alternative name for JB759, while 968 is for JB50",
    """#Table of orthology relatonships for MLP-related genes. Each combination of orthology relationships is summarized by one of the 7 "orthopatterns", representative of the 7 sets in the plotted Venn diagrams.""",
    "#Measurements of adhesion phenotypes in the natural isolates in a tidy format, assembled from the output of yeastmlp, ready for plotting.",
    "#Correlation analysis between flocculation (filtering assay) and gene expression (Clement-Ziza RNA-seq) data.",
    "#Adhesion ratios of mediator deletion strains, assembled from the output of yeastmlp.",
    "#Differentially expressed genes in the srb11 deletion strain from the output of DESeq2.",
    "#Raw results from the deletion screen as measured by yeastmlp.",
    """#Enrichment analysis based on the top 5 percent of adhesive strains identified in the deletion screen.""",
    "#Final results from the 31 high-confidence strains, as measured by yeastmlp.",
    "#Variants identified in candidate genes (from screen and Mediator genes) by short variant analysis in MLP-forming wild isolates.",
    "#List of plasmids used in this study."
]

# Loop through the dictionary and write each dataframe to a separate sheet in the output file
for sheet_name, df, desc in zip(dataframes.keys(), dataframes.values(), descriptions):
    # Add a header to the sheet name
    sheet_name = sheet_name
    title = "# Title: " + sheet_name
    correspondence = "# Correspondence: b.heineike@ucl.ac.uk or j.bahler@ucl.ac.uk"
    
    # Write the dataframe to the sheet
    df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=4)
    
    # Get the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    # Add the title and correspondance to the sheet
    worksheet.write(0, 0, title)
    worksheet.write(1, 0, correspondence)
    worksheet.write(2, 0, desc)

# Save the Excel file
writer.close()


# Now assemble .pdf of all images

In [4]:
!pip install pillow fpdf2
!pip install PyPDF2



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
^C
Traceback (most recent call last):
  File "/Users/k23030440/Library/Python/3.11/bin/pip", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/k23030440/Library/Python/3.11/lib/python/site-packages/pip/_internal/cli/main.py", line 79, in main
    return command.main(cmd_args)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/k23030440/Library/Python/3.11/lib/python/site-packages/pip/_internal/cli/base_command.py", line 101, in main
    return self._main(args)
           ^^^^^^^^^^^^^^^^
  File "/Users/k23030440/Library/Python/3.11/lib/python/site-packages/pip/_internal/cli/base_command.py", line 236, in _main
    self.handle_pip_version_check(options)
  File "/Users/k23030440/Library/Python/3.11/lib/python/site-packages/pip/_internal/cli/req_command.py", line 177, in handle_pip_version_check
    session = se

In [5]:
image_paths = [
    root + "Figures/Figure 1/figure1.png",
    root + "Figures/Figure 2/figure2.png",
    root + "Figures/Figure 3/figure3.png",
    root + "Figures/Figure 4/figure4.png",
    root + "Figures/Figure 5/figure5.png",
    root + "Figures/Figure 6/figure6.png",
    root + "Figures/Supplementary figure 1/suppfig1.png",
    root + "Figures/Supplementary figure 2/suppfig2.png",
    root + "Figures/Supplementary figure 3/suppfig3.png",
    root + "Figures/Supplementary figure 4/suppfig4.png",
    root + "Figures/Supplementary figure 5/suppfig5.png",
    root + "Figures/Supplementary figure 6/suppfig6.png",
    root + "Figures/Supplementary figure 7/suppfig7.png",
    root + "Figures/Supplementary figure 8/suppfig8.png",
    root + "Figures/Supplementary figure 9/suppfig9.png",
    root + "Figures/Supplementary figure 10/suppfig10.png",
    root + "Figures/Supplementary figure 11/suppfig11.png",
    root + "Figures/Supplementary figure 12/suppfig12.png"

]

legends = [
    """Several regulators of MLP formation are conserved between fission and budding yeast, but cell-adhesion effector proteins are not.
(A) Venn diagram of numbers of orthogroups, in which at least one gene in the orthogroup is annotated in GO-terms or phenotypic data related to MLP formation in either S. pombe, S. cerevisiae or C. albicans. Red numbers indicate cell-adhesion proteins. (B) Venn diagram of orthogroups that are conserved across the 3 species (i.e. the middle subset on A) asking whether they are also functionally conserved, i.e. contain at least one gene that is annotated in GO-terms or phenotypic data related to MLP formation in all three species. (C) Functionally conserved genes coloured by their broad functional category as indicated. GO: Gene Ontology, FYPO: Fission Yeast Phenotype Ontology, P: Phenotype annotations.
""",


    """MLP formation in S. pombe natural isolates varies with nutrient conditions and is associated with decreased growth.
(A) Left: Images of the initial observations on the standard laboratory strain JB50 and the natural isolate JB759 showing MLP formation of JB759 in EMM. Cells were grown at 32ºC for 2 days, shaking at 160 rpm. Right: Microscopy images at two magnifications of the JB759 strain grown in EMM for 2 days. (B) Scheme of the high-throughput adhesion assay used to assess MLP formation in S. pombe. (C) Strip plot of adhesion to agar across different conditions in the natural isolate library, with images of representative post-wash agar plates below. Each dot represents the mean adhesion value for a given strain in a specific condition. The red dashed line shows a cut-off for strong phenotypes (intensity after wash >0.2 times than before wash). Each condition was compared to rich media (YES) with the null hypothesis that they do not increase MLP formation. P-values were obtained using a one-sided permutation-based T-test and Bonferroni correction. Comparisons were marked not significant (ns.) where the null hypothesis could not be rejected at significance threshold 0.05. Strains around the edges were not taken into account for any statistical analysis (see Methods). The lab strain JB50 and natural isolate JB759 used in panel (A), as well as the lab strain, JB22, and the highly adhesive strains JB914 and JB953, are highlighted in conditions in which they are adhesive. (D) Histogram showing the number of unique strains forming MLPs in a given number of conditions. (E) Panel of light microscopy images for JB953 and JB914 in EMM and YES, showing representative images of MLP-formation. Scale bars are 100 µm. Rest of the microscopy images from this experiment are displayed in Supplementary Figure 4. Representative images of culture flasks are shown in Supplementary Figure 5.  (F) Scatter plot of mean cell densities (measured in normalised inverse pixel intensity) before and after washing. Each dot represents one strain in one condition, orange dots represent adhesive data points (ratio of before-wash to after-wash intensity >0.2, dashed red line) and purple dots represent non-adhesive data points. The histogram shows the distribution of cell densities before washing as a proxy for growth. The vertical dotted orange and purple lines mark the mean pre-wash densities for the two populations.
""",


       """MLP formation in JB759 is driven by mbx2 expression and associated with a single-nucleotide deletion on chromosome II.
(A) Scheme for the JB759xJB50 segregant library. Red and blue stripes indicate genomic recombination resulting from meiosis. SNPs for the strains inside the grey box were previously identified (49) and genome sequencing data for the strains in the light blue box were generated in this work (Methods). (B) (Left) Example plate scans of segregants grown on EMM or YES after washing, shown in viridis colourmap. (Right) Adhesion to agar of segregant strains on EMM (mean of 10 replicates) compared to YES (mean of 2 replicates), along with significance of the difference obtained using permutation-based T-test. (C) Correlations of mbx2 and flocculin gene expression with flocculation in EMM. Each dot represents a strain from the segregant library. (D) (Left) Box plot of adhesion measurements overlaid comparing adhesion measurements from control, GFP overexpression and mbx2 overexpression strains in both JB22 and JB50 backgrounds, as well as measurements for empty media (Methods). (Right) Representative images of the flocculating mbx2 overexpression strain in YES and EMM. Rest of the flask images are found in Supplementary Figure 7B. (E) (Left) Manhattan plot of QTL analysis results for flocculation in EMM. The red line shows the Bonferroni threshold, while the green dashed line shows the highest possible significance achievable using 20,000 permutations. (Middle) Candidate variant II:2316851 TA>T is associated with increased flocculation in EMM (filtering assay, mean of 3 replicates). (Right) Candidate variant II:2316851 TA>T is associated with increased adhesion on EMM (washing assay). P-values determined using a permutation-based T-test with 1E+6 permutations. (F) Microscopy image of srb11ΔCRISPR in EMM showing flocculation. The inset shows a close-up of the region highlighted in box “a”.
""",


"""Cdk8 kinase module deletions upregulate mbx2 in minimal media, but not in rich media, and lead to MLP formation.
(A) Scheme showing how a single nucleotide deletion leads to frameshift and premature stop-codon in srb11. (B) Full Srb11 structure (green) compared with the truncated Srb11 structure (red) as predicted using Colabfold (92). Scheme on the right shows Srb11 in the context of the Cdk8 kinase module of the Mediator complex. The structure was sketched based on structural data (84), and functional roles were summarised based on (84,86). (C) Box plot showing adhesion values from Mediator gene deletion strains from the genome-wide prototrophic deletion library on EMM and YES as indicated. Each dot represents a replicate. P-value represents comparison of each strain against the least adhesive strain in the assay (med27Δ) using a T-test. (D) Differential expression analysis of segregant strains split on the II:2316851 TA>T single-nucleotide deletion. Fold-change values and P-values were obtained from DESeq2 (67). Flocculins and mbx2 are highlighted in red. (E) Overlap of upregulated genes in three CKM mutants based on our srb11 data and data from Linder et al. (68). (F) Simplified model for how CKM deletions lead to MLP formation. (G) Adhesion measurements for the srb11Δ strain obtained from the deletion collection, and its derived strain after mbx2 knock-out with CRISPR. Each dot represents a replicate measurement. Error bars represent the 95% confidence interval for the mean. (H) RT-qPCR showing mbx2 expression in JB22 (WT), srb10Δ::Kan, srb11Δ strains obtained from the genome-wide prototrophic deletion collection in EMM or YES. Expression of each gene is normalised relative to act1 expression. Height of each bar reflects the mean of three biological replicates which are indicated by dots.
""",


"""Deletion library screen identified 31 genes associated with MLP formation on minimal media.
(A) Scatter plot of mean cell densities before and after washing. Each dot represents a deletion strain, and colours represent adhesive and non-adhesive strains as indicated. The red line represents the cut-off at the 95th percentile of adhesion values (Supp Fig 10B). The histogram shows the distribution of cell densities before washing as a proxy for growth. The dotted orange and purple lines mark the mean pre-wash intensity values for the two populations. The P-value was determined using a permutation-based T-test with 1E+5 permutations. (B) Bar plot showing fold enrichment for the top 10 most significantly enriched processes from AnGeLi (74). Blue circles indicate terms associated with ribosomes, and orange circles indicate terms associated with decreased growth. Gene sets were sorted based on P-values, with increasing colour intensity representing increasing -log10(P-value). The size of the circle at the end of each bar represents the size of the gene set. GEX: Gene expression; pop.: population. (C) Box plots of adhesion ratios obtained with the washing assay for 4 of the 31 verified hits (rpl15Δ, rpa12Δ, fkh2Δ, SPNCRNA.1234Δ) on EMM (light blue) vs YES (orange). Each dot is an independent observation. Remaining hits are shown in Supp Fig 11. (D) Venn diagram showing functional conservation of the genetically conserved hits from our screen. The hits SPAC607.02c, sre2, meu27, for3 are not shown as they are not genetically conserved (only present in S. pombe). Only fkh2 and res1 are also annotated as being involved in MLP formation in S. cerevisiae and C. albicans. Annotations are assigned using both GO and Phenotype data (GO+P). (E) Scatter plot of adhesion ratios and before-wash colony intensities overlaid by a histogram showing before-wash colony intensities of non-adhesive deletion strains, which were assayed in the middle 60 spots during the original screen. The green-shaded area marks strains that are above the lowest 5th percentile of before-wash colony intensity in the non-adhesive strains, a proxy for growth rate. The srb10/11Δ strains, highlighted in green, are the most adhesive strains among those with growth values above the lowest 5th percentile. (F) Line plot of growth measurements (OD600) from WT (JB22), empty media, srb11Δ, srb10Δ, mus7Δ, rpa12Δ, kgd2Δ and tlg2Δ strains in EMM and YES.
""",


"""Model for EMM-dependent MLP formation of CKM mutants.
We propose that the CKM phosphorylates Mbx2 and targets it for degradation, based on similar observations in C. albicans (88) and S. cerevisiae (90). Additionally, mbx2 expression is repressed through an unknown mechanism in YES, as we observed using RT-qPCR. In minimal media, if members of the CKM are deleted, mbx2 becomes upregulated, triggering the expression of flocculin genes, which in turn cause MLP formation. Deletion of ribosomal genes also triggers MLP formation, although it is unclear whether this occurs through upregulation of mbx2 or directly through the flocculin genes. The red arrows indicate our main findings, and the red question marks highlight the main outstanding mechanistic questions. See main text for details.
""",


"""Plate-reader assay and filtering assay for flocculation are correlated.
Scatter plot comparison of flocculation for the JB50-JB759 segregant library by measuring each strain with a conventional filtering assay (x-axis, n=3) and our high-throughput plate-reader assay which measures the CV of OD600 within each well (y-axis, n=5) (Methods). Each point represents the mean flocculation values for a strain. P-value was obtained with a one-tailed test on whether the Spearman correlation coefficient is larger than 0.
""",


"""Conservation of MLP-related orthogroups between S. pombe, S. cerevisiae and C. albicans.
(A) Venn diagram representing genetic and (B) functional conservation of orthogroups between the three species using data only from GO-terms. (C) Highest alignment scores for either 25 protein sequences annotated as unique to S pombe or 50 random conserved sequences using either BLAST-P or Foldseek to identify similar proteins in the S. cerevisiae or C. albicans proteome (Methods). Each dot represents the highest score using the indicated search algorithm for a given protein. P-values were derived from Mann-Whitney U tests comparing top scores for conserved proteins with either all unique proteins (black) or just unique S. pombe flocculins (green).
""",


"""Further analysis on adhesion phenotypes of the natural isolates across different conditions.
(A) Scatter plot comparison of flocculation and adhesion for the natural isolate library by measuring each strain with a high-throughput plate-reader assay (x-axis, n=11) and our high-throughput adhesion assay (y-axis, n=10). Each point represents the mean values for a strain with the standard error of the mean as error bars. Stars mark JB759 (red) and the lab strain JB50 (blue). (B) Elbow plot of the number of strains designated as “MLP-forming” in at least one condition (y-axis) given different cut-off thresholds (x-axis). Empirically we found that 0.2 sits in the elbow of the plot, suggesting that it is a reasonable cut-off. (C) Phosphate starvation induces MLP formation in the lab strain JB50. Images of the JB50 strain growing in 25ml flasks with 10ml media of either EMM (left) or EMM-P (right). Strains were first grown up on YES plates and then transferred to liquid cultures. There are clearly visible flocs forming in EMM-P. (D) Bar plot of log2FC in gene expression during 2 days of phosphate starvation for selected genes, including the transcription factor mbx2 and various flocculins. Data from (81). Red bars represent significant log2FC values.
""",


"""Microscope images of wild isolates exhibiting MLPs.
Images of during exponential growth of JB50 (lab strain) and three wild isolates (JB759, JB914, and JB953) in media conditions under which they exhibit MLPs. Note: EMM-P images were taken with a 20x objective with 1x zoom, while EMM and YES images were taken with a 5x objective and 2x optical zoom. Images were scaled to be comparable. The images of JB914 and JB953 in EMM and YES are duplicated from this panel and also included in Figure 2. Scale bars are 100 µm.
""",

"""Culture flask images of wild isolates exhibiting MLPs.
Images of during (A) exponential growth and (B) in saturated cultures after two days of JB50 (lab strain) and three wild isolates (JB759, JB914, and JB953) in media conditions under which they exhibit MLPs. 
""",


"""MLP formation in S. pombe involves mbx2 and CKM components.
(A) Scatter plot comparison of flocculation and adhesion for the JB50-JB759 library by measuring each strain with the filtering assay (x-axis, n=3) and the high-throughput adhesion assay (y-axis, n=10). Each point represents the mean values for a strain with the standard error of the mean as error bars. Stars mark the two parental strains JB759 (red) and JB50 (blue). There is a strong correlation between the two MLPs. (B) Scatter plot of expression levels of four flocculin genes against mbx2 expression in the JB50-JB759 segregant library (data from (49)), demonstrating a strong association. (C) (Left) Images showing flocculating cultures of srb10 and srb11 in 250ml flasks grown in EMM. (Right) Representative microscopy image of srb11∆::Kan (D) Barplot of adhesion values showing that the Kan deletion of srb11 found in the deletion library phenocopies the CRISPR deletion of srb11. Each dot is an independent measurement. Error bars represent the 95% confidence interval for the mean. (E) Box plot showing adhesion values from validation cohort of fresh Mediator gene deletion strains on EMM and YES as indicated. Each dot represents a replicate.
""",


"""Overexpression of mbx2 drives MLP formation. 
(A) Microscopy and (B) flask images illustrating the effects of mbx2 overexpression on cells grown in EMM or YES for both the h- (JB22) and h90 (JB50) backgrounds and the h90 JB50 background. Flask images were from overnight cultures, and microscope images were from overnight cultures after dilution into fresh media for two hours. The flask images of prRPL2102-mbx2 (JB22) are duplicated and also shown in Figure 3. Scale bars are 75 µm.
""",


"""Mbx2 drives MLP formation in the srb11Δ strain.
(A) Venn diagram showing the intersection of three gene sets: (i) Genes upregulated in CKM mutants, meaning upregulated in the med12Δ and srb10Δ microarray data from (68) and in the srb11 truncated segregant RNA-seq data from (49) (Fig 4E), (ii) Genes upregulated after mbx2 overexpression obtained from (27) and (iii) Genes bound by Mbx2 obtained from (27). (B) Images showing non-flocculating cultures of three biological replicates of srb11/mbx2 double deletion strains. (C) Empirical cumulative distribution function of L1 distances between phenotype vectors for the srb11Δ strain and all other gene deletion strains for growth phenotypes across 131 different conditions. For each condition in each strain, a number of -1, 0, or 1 was assigned to denote sensitivity, neutrality and resistance, and the L1 distance is the sum of the absolute value of the differences across all phenotypes as measured in (96). The 10 deletion strains closest in their phenotypes to srb11Δ and med13Δ are highlighted on the plot. The similarity score of med13Δ is in the 21st percentile. (D) Scatter plot of fold changes comparing two different data sources for transcriptomic changes upon fkh2 deletion. Dots mark individual genes in the datasets, while the line of best fit is shown in red. This plot concludes that it is fair to use the two data sources to draw joint conclusions from them. (E) Bar plot showing log2 fold-changes in mbx2 expression levels from experiments in which fkh2 was deleted (red) and in which CKM subunits are deleted (blue). Selected fkh2 deletion experiments included microarray data from (86) and (97), while CKM subunit deletion experiments included med12Δ and srb10Δ microarray data from (68), fold change calculated by grouping RNA-seq data from (48) for segregants with the srb11 truncation vs those without the mutation (Fig 3D), and the RT-qPCR data collected in this work. (F) Bar plot showing log2 fold-changes in gsf1 expression in YES compared to EMM. There is no significant upregulation in either of the two replicates. Data and P-values were taken from (98).
""",


"""The srb11 variant is present in the JB759 and JB914 natural isolates, and segregates with MLP-formation across natural isolates closely related to those strains.
(A) S. pombe reference genome visualised for each of the three chromosomes, with vertical lines marking variant loci (called in (45)) compared to the reference genome in at least one of the two strains. Variant loci with the same alternative allele for JB759 and JB914 are marked with purple, while variant loci where alleles do not match are marked with orange. (B) Inset showing the region surrounding the causal srb11 variant. The +/- 20Kb window is highlighted. Vertical lines indicate variant alleles as in A. (C) Dendrogram illustrating genomic similarity of natural isolates based on the +/- 20Kb window around the srb11 frameshift mutation. Red text indicates strains closely related to JB759 considering the whole genome (based on (45)), while blue text indicates strains related to JB914. Heatmap below indicates adhesion ratio in EMM.
""",


"""Deletion library screen for MLP formation on EMM.
(A) Histogram showing cell densities, measured by inverse pixel intensity after 4 days of growth, for strains in our deletion library screen before washing. (B) Empirical cumulative distribution function (ECDF) of adhesion ratios from the deletion library screen, with red lines showing the cut-off for the 95th percentile. Strains above the cut-off were used for enrichment analysis.
""",

"""Verified hits from the deletion library screen for MLP formation, assayed both on EMM and YES.
Box plots of adhesion ratios obtained with the washing assay for the 31 high-confidence hits on EMM (light blue) vs YES (orange). Each dot is an independent observation. Adhesion was generally specific to EMM, except in the case of sre2Δ. The images of rpl15Δ, rpa12Δ, fkh2Δ, SPNCRNA.1234Δ are duplicated from this panel and also included in Figure 5.
""",

"""MLP-forming hits belong in various phenotypic clusters, but generally exhibit slow growth.
(A) Bar plot showing the number of MLP-forming (blue, left axis scale) and total (blue, right axis scale) strains in each broad phenotypic cluster of deletion strains identified in (96). (B) Histogram of mean fitness values, based on growth in solid EMM media, for each strain as defined in (96). MLP-forming hit strains are shown in purple, and all other strains are in yellow. Blue and red dotted lines show the thresholds beyond which strains were defined as slow- or fast-growing, respectively. 
"""

]
output_path = root + "Supplementary material/figures.pdf"


In [6]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None 
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Image as ReportLabImage, Paragraph, PageBreak
from reportlab.lib.styles import ParagraphStyle
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Image as ReportLabImage
import os
import PyPDF2


In [7]:
def compress_pdf(pdf_path):
    compressed_path = pdf_path.replace(".pdf", "_compressed.pdf")
    with open(pdf_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pdf_writer = PyPDF2.PdfWriter()

        for page in pdf_reader.pages:
            pdf_writer.add_page(page)

        with open(compressed_path, "wb") as compressed_pdf:
            pdf_writer.write(compressed_pdf)

    os.remove(pdf_path)  # Remove the original uncompressed PDF
    os.rename(compressed_path, pdf_path)  # Rename the compressed PDF to original name


In [8]:
from reportlab.lib.pagesizes import A4, landscape
from reportlab.platypus import SimpleDocTemplate, Image as ReportLabImage, Paragraph, PageBreak
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus.doctemplate import NextPageTemplate, PageTemplate
from reportlab.platypus.frames import Frame
from PIL import Image

def create_pdf_from_images(image_paths, output_path, n_main_figs, n_supp_figs, legends, landscape_figures=None):
    """
    Create a PDF from images with support for landscape orientation on specific figures.
    
    Parameters:
    - image_paths: list of image file paths
    - output_path: output PDF file path
    - n_main_figs: number of main figures
    - n_supp_figs: number of supplementary figures
    - legends: list of legend text for each figure
    - landscape_figures: list of figure indices (0-based) that should be in landscape mode
    """
    if landscape_figures is None:
        landscape_figures = []
    
    # Create document with templates for both orientations
    doc = SimpleDocTemplate(output_path, pagesize=A4, rightMargin=10, leftMargin=10, topMargin=10, bottomMargin=10)
    
    # Define page templates
    portrait_frame = Frame(10, 10, A4[0]-20, A4[1]-20, id='portrait')
    landscape_frame = Frame(10, 10, landscape(A4)[0]-20, landscape(A4)[1]-20, id='landscape')
    
    portrait_template = PageTemplate(id='portrait', frames=[portrait_frame], pagesize=A4)
    landscape_template = PageTemplate(id='landscape', frames=[landscape_frame], pagesize=landscape(A4))
    
    doc.addPageTemplates([portrait_template, landscape_template])
    
    story = []
    
    for i, (image_path, legend) in enumerate(zip(image_paths, legends)):
        # Determine if this figure should be in landscape
        is_landscape = i in landscape_figures
        
        # Switch to appropriate page template
        if is_landscape:
            story.append(NextPageTemplate('landscape'))
            page_width = landscape(A4)[0] - 20  # Account for margins
            page_height = landscape(A4)[1] - 20
            max_width = page_width - 50  # Leave some margin for text
            max_height = page_height - 150  # Leave space for caption
        else:
            story.append(NextPageTemplate('portrait'))
            page_width = A4[0] - 20
            page_height = A4[1] - 20
            max_width = 550  # Original width for portrait
            max_height = 700  # Original max height for portrait
        
        # Load and resize image
        img = Image.open(image_path)
        width, height = img.size
        aspect_ratio = height / float(width)
        
        if is_landscape:
            # For landscape pages, optimize sizing differently
            # Start with width-based sizing
            new_width = max_width
            new_height = new_width * aspect_ratio
            
            # If height exceeds limit, scale down based on height
            if new_height > max_height:
                new_height = max_height
                new_width = new_height / aspect_ratio
        else:
            # Original portrait sizing logic
            new_width = max_width
            new_height = new_width * aspect_ratio
            
            if new_height > 700:
                new_height = 750
                new_width = new_height / aspect_ratio
            elif new_height > 600:
                new_height = 650
                new_width = new_height / aspect_ratio
        
        # Create ReportLab image
        reportlab_img = ReportLabImage(image_path, width=new_width, height=new_height)
        
        # Create captions
        if i < n_main_figs:
            caption1 = f"<strong>Figure {i + 1}</strong>"
            caption2 = f"<strong>Figure {i + 1}</strong>: {legend}"
        elif i < n_main_figs + n_supp_figs:
            caption1 = f"<strong>Supplementary Figure {i - n_main_figs + 1}</strong>"
            caption2 = f"<strong>Supplementary Figure {i - n_main_figs + 1}</strong>: {legend}"
        else:
            caption1 = ""
            caption2 = ""
        
        caption_style1 = ParagraphStyle(name='Caption', fontSize=15, leading=20)
        caption_style2 = ParagraphStyle(name='Caption', fontSize=9, leading=16)
        
        # Add content to story
        if caption1:
            story.append(Paragraph(caption1, caption_style1))
            story.append(Paragraph("<br/>", caption_style2))
        
        story.append(reportlab_img)
        
        if caption2:
            story.append(Paragraph(caption2, caption_style2))
        
        story.append(PageBreak())
    
    # Build the document
    doc.build(story)
    compress_pdf(output_path)

# Example usage:
# Figures 2 and 5 will be in landscape mode (0-based indexing: 1 and 4)
# create_pdf_from_images(image_paths, output_path, 7, 7, legends, landscape_figures=[1, 4])


create_pdf_from_images(image_paths, output_path, 6,12, legends, landscape_figures=[])
